In [13]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

import mar
import preprocessing

In [2]:
data_dir = os.path.join(os.path.dirname(os.getcwd()), "data")
pat_num = 9

In [9]:
mar_data = preprocessing.load_mar_data(data_dir, pat_num)
sickbay_data = preprocessing.load_sickbay_data(data_dir, pat_num)
sbs_data = preprocessing.load_retro_data(data_dir, pat_num, ignore_default=True)

In [10]:
accel_data = preprocessing.process_accel_data(data_dir, pat_num, save=os.path.join(data_dir, f'Patient{pat_num}', f'accel.csv'))

hello world


In [11]:
ecg_data = preprocessing.load_ecg_data(data_dir, pat_num)

In [ ]:
mar_narcotics, mar_paralytics, mar_alpha_agonists, mar_ketamines, mar_propofols, mar_etomidates, mar_benzodiazepines = mar.filter_mar(mar_data)
drug_classes = [mar_narcotics, mar_paralytics, mar_alpha_agonists, mar_ketamines, mar_propofols, mar_etomidates, mar_benzodiazepines]

for drug_class in drug_classes:
    print(drug_class.shape)

(414, 4)
(0, 4)
(44, 4)
(9, 4)
(41, 4)
(0, 4)
(290, 4)


In [ ]:
drug_doses = []
drug_concentrations = []

for drug_class in drug_classes:
    if drug_class.shape[0] > 0:
        